In [3]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from dataclasses import dataclass

load_dotenv()

True

RUNTIME IN TOOL

In [ ]:
from langchain.tools import tool, ToolRuntime

@dataclass
class Context:
    user_name: str

@tool
def get_name(runtime: ToolRuntime[Context]):
    '''Get the name of user '''
    print(runtime)
    print(type(runtime))         # <class 'langgraph.runtime.Runtime'>
    print(runtime.context)       # Context(user_id='123')
    print(runtime.store)         # PostgresStore(...)
    print(id(runtime)) 
    print(runtime.state)
    # You can also access the store for long-term memory
    # preferences = "Default preferences"
    # user_name = runtime.context.user_name    
    # if runtime.store:
    #     memory = runtime.store.get(("users",), user_name)
    #     if memory:
    #         preferences = memory.value["preferences"]
    # print(runtime.store)  
    # return f"User {user_name} preferences: {preferences}"
    return runtime.context.user_name         

agent = create_agent(
    model= "gpt-4o",
    tools = [get_name],
    context_schema=Context
)

In [20]:
res = agent.invoke(
    {"messages": [{"role": "user", "content": "What's my name?"}]},
    context=Context(user_name="Aayushmaan")  
)
res["messages"][-1].content

None


'Your name is Aayushmaan.'

RUNTIME IN MIDDLEWARE

In [ ]:
from dataclasses import dataclass
from langchain.messages import AnyMessage
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import dynamic_prompt, ModelRequest, before_model, after_model
from langgraph.pregel.main import Output
from langgraph.runtime import Runtime

@dataclass
class Contex:
    user_name: str


# dynamic prompt
@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest):
    user_name = request.runtime.context.user_name
    system_prompt = f"You are helpful assistant. Adress the user as {user_name}"
    return system_prompt

"""
NOTE: Understanding Hook Parameters

You might wonder: "Why don't we just use ModelRequest everywhere to access runtime and state?"

Great question! Each hook receives different parameters because they serve different purposes:

**before_model Hook:**
- Runs BEFORE the request is built
- The ModelRequest doesn't exist yet
- We use `state` and `runtime` separately because we're still preparing things
- Think of it as: "Getting ready before making the call"

**after_model Hook:**
- Runs AFTER the model has responded
- The ModelRequest is already gone (it was consumed by the model)
- Now we have the ModelResponse in our state
- We use `state` and `runtime` because we're working with the OUTPUT, not the input
- Think of it as: "Processing what we got back"

**@wrap_model_call or @dynamic_prompt(uses ModelRequest):**
- Intercepts the ACTUAL model call
- The ModelRequest exists at this moment
- We can modify what goes INTO the model right before it's sent
- Think of it as: "Last chance to edit before sending"

Simple rule: 
If you need to modify the REQUEST → use @wrap_model_call or @dynamic_prompt or other middlewares which excpets Modelrequest
If you need to manage lifecycle → use before_model/after_model
"""
# Before model hook
@before_model
def log_before_model(state: AgentState, runtime: Runtime[Context]) -> dict | None:  
    print(f"Processing request for user: {runtime.context.user_name}")  
    return None

# After model hook
@after_model
def log_after_model(state: AgentState, runtime: Runtime[Context]) -> dict | None:  
    print(f"Completed request for user: {runtime.context.user_name}")  
    return None



In [62]:
agent = create_agent(
    model="gpt-4o",
    tools=[],
    middleware=[dynamic_system_prompt, log_before_model, log_after_model],  
    context_schema=Context
)


In [63]:
res = agent.invoke(
    {"messages": [{"role": "user", "content": "What's my name?"}]},
      context=Context(user_name="Max Verstappen")
    )

res["messages"][-1].content

Processing request for user: Max Verstappen
Completed request for user: Max Verstappen


'Your name is Max Verstappen.'